In [2]:
cd ..

/home/john/Development/python/rpyc_docker


In [3]:
import logging
logger = logging.getLogger("rpyc_docker")
logger.setLevel(logging.INFO)

In [4]:
from docker import Client
docker = Client(base_url='unix://var/run/docker.sock')

In [5]:
import rpyc_docker.worker
reload(rpyc_docker.worker)
import rpyc_docker.rpyc_browser_worker
reload(rpyc_docker.rpyc_browser_worker)
import rpyc_docker.manager
reload(rpyc_docker.manager)
from rpyc_docker import BrowserRpycWorker
from rpyc_docker import Manager

In [6]:
cd examples/

/home/john/Development/python/rpyc_docker/examples


# Create Workers for fetching results from reddit and duckduckgo

In [7]:
import time
from reddit_page import RedditPage

class RedditWorker(BrowserRpycWorker):
    def __init__(self,docker,subReddit,redditResultQueue):
        BrowserRpycWorker.__init__(self,docker)
        self.subReddit = subReddit
        self.redditResultQueue = redditResultQueue
        
    def work(self):
        self.create_container()
        time.sleep(2)
        self.connect_rpyc()
        self.setup_browser("firefox")
        redditPage = RedditPage(self.browser)
        redditPage.goto_subreddit(self.subReddit)
        results = redditPage.get_reddit_titles()
        self.redditResultQueue.put([self.subReddit,results])
        return True

In [8]:
import time
from duckduckgo_page import DuckDuckGoPage

class DuckDuckGoWorker(BrowserRpycWorker):
    def __init__(self,docker,searchTerm,resultQueue):
        BrowserRpycWorker.__init__(self,docker)
        self.searchTerm = searchTerm
        self.resultQueue = resultQueue
        
    def work(self):
        self.create_container()
        time.sleep(2)
        self.connect_rpyc()
        self.setup_browser("firefox")
        duckDuckGoPage = DuckDuckGoPage(self.browser) 
        duckDuckGoPage.search(self.searchTerm)
        results = duckDuckGoPage.get_results()
        self.resultQueue.put([self.searchTerm,results])
        return True

In [9]:
cd ..

/home/john/Development/python/rpyc_docker


#using manager to run a reddit workers

In [8]:
import Queue

In [9]:
redditResultQueue = Queue.Queue()

In [10]:
workQueue = Queue.Queue()

In [12]:
redditQueries = ["/r/python","/r/clojure","r/programming","r/javascript"]

In [11]:
for redditQuery in redditQueries :
    workQueue.put([RedditWorker,[docker,redditQuery,redditResultQueue],{}])

In [12]:
manager = Manager(workQueue,1)

In [13]:
manager.start()

INFO:rpyc_docker:<class '__main__.RedditWorker'>,[<docker.client.Client object at 0x7fa328704d10>, '/r/python', <Queue.Queue instance at 0x7fa31aed9a28>],{}


In [20]:
manager.report()

'WorkerManager Report\nRunning workers 0 \nResults 4 \nErrors 0 \nQueue Size 0'

In [21]:
result  = redditResultQueue.get_nowait()

#Using manager to run Duckduckgo workers

In [1]:
import Queue

In [10]:
duckDuckGoResultQueue = Queue.Queue()

In [11]:
duckDuckGoQueries = ["python","clojure","programming","javascript"]

In [13]:
workQueue = Queue.Queue()

In [14]:
for duckDuckGoQuery in duckDuckGoQueries :
    workQueue.put([DuckDuckGoWorker,[docker,duckDuckGoQuery,duckDuckGoResultQueue],{}])

In [15]:
manager = Manager(workQueue,1)

In [16]:
manager.start()

INFO:rpyc_docker:<class '__main__.DuckDuckGoWorker'>,[<docker.client.Client object at 0x7fd9340b5c50>, 'python', <Queue.Queue instance at 0x7fd92d096290>],{}


In [27]:
manager.report()

'WorkerManager Report\nRunning workers 0 \nResults 2 \nErrors 2 \nQueue Size 0'

In [26]:
manager.errors[0][0].splitlines()

[u'Traceback (most recent call last):',
 u'  File "rpyc_docker/worker.py", line 38, in run',
 u'    self.result = self.work()',
 u'  File "<ipython-input-8-b699811c2d6e>", line 17, in work',
 u'    results = duckDuckGoPage.get_results()',
 u'  File "duckduckgo_page.py", line 38, in get_results',
 u'  File "/usr/local/lib/python2.7/dist-packages/rpyc/core/netref.py", line 196, in __call__',
 u'    return syncreq(_self, consts.HANDLE_CALL, args, kwargs)',
 u'  File "/usr/local/lib/python2.7/dist-packages/rpyc/core/netref.py", line 71, in syncreq',
 u'    return conn.sync_request(handler, oid, *args)',
 u'  File "/usr/local/lib/python2.7/dist-packages/rpyc/core/protocol.py", line 441, in sync_request',
 u'    raise obj',
 u'WebDriverException: Message: resultElm.querySelector(...) is null',
 u'Stacktrace:',
 u'[',
 u'u',
 u"'",
 u' ',
 u' ',
 u' ',
 u' ',
 u'a',
 u't',
 u' ',
 u'a',
 u'n',
 u'o',
 u'n',
 u'y',
 u'm',
 u'o',
 u'u',
 u's',
 u'/',
 u'<',
 u' ',
 u'(',
 u'h',
 u't',
 u't',
 u

#Running two different docker workers at once

In [29]:
duckDuckGoResultQueue = Queue.Queue()
redditResultQueue = Queue.Queue()

In [30]:
workQueue = Queue.Queue()
for duckDuckGoQuery,redditQuery in zip(duckDuckGoQueries,redditQueries):
    workQueue.put([DuckDuckGoWorker,[docker,duckDuckGoQuery,duckDuckGoResultQueue],{}])
    workQueue.put([RedditWorker,[docker,redditQuery,redditResultQueue],{}])

In [31]:
manager = Manager(workQueue,2)

In [32]:
manager.start()

INFO:rpyc_docker:<class '__main__.DuckDuckGoWorker'>,[<docker.client.Client object at 0x7fd9340b5c50>, 'python', <Queue.Queue instance at 0x7fd917f251b8>],{}
INFO:rpyc_docker:<class '__main__.RedditWorker'>,[<docker.client.Client object at 0x7fd9340b5c50>, '/r/python', <Queue.Queue instance at 0x7fd917f25128>],{}


In [34]:
manager.report()

'WorkerManager Report\nRunning workers 0 \nResults 6 \nErrors 2 \nQueue Size 0'